In [1]:
import numpy as np
import math
import time
import xlrd
import matplotlib.pyplot as plt
from data_extract import *
from data_preprocess import *


# Data Extraction

In [2]:
gdf_fname = "C:/Users/Liangyu/Desktop/BME725/Final Project/Data/S01_ME/motorexecution_subject1_run1.gdf"
compressed_fname = "S1R1ME.bz2"

t1 = time.time()
HDR, data = read_data(gdf_fname)
seqs_v_class_map = segregate_data_into_classes(HDR, data)
#compress_segregated_data(seqs_v_class_map, compressed_fname)
print("Done processing file in %f s\n" % (time.time() - t1))

# define sampling rate
fs = int(HDR.SampleRate/2)

Finished reading header
Read header of C:/Users/Liangyu/Desktop/BME725/Final Project/Data/S01_ME/motorexecution_subject1_run1.gdf in 0.005981 s

Finished reading data
Read data of C:/Users/Liangyu/Desktop/BME725/Final Project/Data/S01_ME/motorexecution_subject1_run1.gdf in 46.872737 s

Finished segregating data into classes in 0.653271 s

Done processing file in 47.531990 s



# Window the 1D eeg vector data into time bins

In [3]:
# window size parameters
width = int(fs/4)
nOverlap = int(fs/8)
initTime = 0
finTime = int(fs*3)-nOverlap

# segment the data based into windows
seqs_v_class_map_binned = data_segmentation(seqs_v_class_map, initTime, finTime, width, nOverlap)

# show the old and new array dimensions for class 1
print(np.shape(seqs_v_class_map[1]))
print(np.shape(seqs_v_class_map_binned[1]))

(6, 769, 64)
(6, 23, 64, 64)


# Convert 1D eeg vector to 2D spatial array

In [4]:
# define the mapping of each channel to the r,c values of the mesh (This needs to be changed. I'ts currently just in order)
maps = [(0, 4), (0, 5), (0, 6),\
        (1, 3), (1, 4), (1, 5), (1, 6), (1, 7),\
        (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9),\
        (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9),\
        (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10),\
        (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9),\
        (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9),\
        (7, 3), (7, 4), (7, 5), (7, 6), (7, 7),\
        (8, 4), (8, 5), (8, 6),\
        (9, 5)]
 
# Give the location of the file that maps eeg index to channel name 
loc = ("C:/Users/Liangyu/Desktop/BME725/Final Project/ChannelLabel.xlsx") 

# Open the xls file
wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(0) 

# create the mesh
mesh = [ [ "   " for y in range(0,11) ] for x in range(0,10) ] 
for chan in range(1,sheet.nrows):
    mesh[maps[chan][0]][maps[chan][1]] = sheet.cell_value(chan, 1)

# print the 2D mesh of channels
for x in range(0,10): 
    print(mesh[x])
    
# change the 1D eeg signal to 2D eeg signal based on the defined map
seqs_v_class_map_mesh2 = data_1D_to_2D(seqs_v_class_map, 10, 11, maps)
# print the size of the new array for the first class
print(np.shape(seqs_v_class_map_mesh2[1]))

['   ', '   ', '   ', '   ', '   ', 'F3 ', 'F1 ', '   ', '   ', '   ', '   ']
['   ', '   ', '   ', 'Fz ', 'F2 ', 'F4 ', 'FFC5h ', 'FFC3h ', '   ', '   ', '   ']
['   ', 'FFC1h ', 'FFC2h ', 'FFC4h ', 'FFC6h ', 'FC5 ', 'FC3 ', 'FC1 ', 'FCz ', 'FC2 ', '   ']
['   ', 'FC4 ', 'FC6 ', 'FTT7h ', 'FCC5h ', 'FCC3h ', 'FCC1h ', 'FCC2h ', 'FCC4h ', 'FCC6h ', '   ']
['FTT8h ', 'C5 ', 'C3 ', 'C1 ', 'Cz ', 'C2 ', 'C4 ', 'C6 ', 'TTP7h ', 'CCP5h ', 'CCP3h ']
['   ', 'CCP1h ', 'CCP2h ', 'CCP4h ', 'CCP6h ', 'TTP8h ', 'CP5 ', 'CP3 ', 'CP1 ', 'CPz ', '   ']
['   ', 'CP2 ', 'CP4 ', 'CP6 ', 'CPP5h ', 'CPP3h ', 'CPP1h ', 'CPP2h ', 'CPP4h ', 'CPP6h ', '   ']
['   ', '   ', '   ', 'P3 ', 'P1 ', 'Pz ', 'P2 ', 'P4 ', '   ', '   ', '   ']
['   ', '   ', '   ', '   ', 'PPO1h ', 'PPO2h ', '   ', '   ', '   ', '   ', '   ']
['   ', '   ', '   ', '   ', '   ', '   ', '   ', '   ', '   ', '   ', '   ']
(6, 769, 10, 11)


# Window the 2D eeg array data into time bins

In [5]:
# window size parameters
width = int(fs/4)
nOverlap = int(fs/8)
initTime = 0
finTime = int(fs*3)-nOverlap

# segment the data based into windows
seqs_v_class_map_binned2 = data_segmentation(seqs_v_class_map_mesh2, initTime, finTime, width, nOverlap)

# show the old and new array dimensions for class 1
print(np.shape(seqs_v_class_map[1]))
print(np.shape(seqs_v_class_map_binned2[1]))

(6, 769, 64)
(6, 23, 64, 10, 11)
